In [ ]:
import pandas as pd

In [ ]:
folder = "../run/data/Norfolk"
dir_save = "../run/data/stream_data"

In [ ]:
df_meta = pd.read_csv(folder + "/Norfolk_metadata.csv")
df_meta

In [ ]:
# check list of fault type and intensities
df_meta.sensor_filename.unique()

In [ ]:
# create empty scenario json
json_scenario = {}

In [ ]:
# add scenario 1
json_scenario['1'] = {
    "name": "baseline",
    "date_start": "2006-05-01",
    "date_end": "2006-05-31",
    "filename":False,
}

# add scenario 2
json_scenario['2'] = {
    "name": "improperly_sized_equipment_wshp_1.5",
    "date_start": "2006-06-01",
    "date_end": "2006-07-31",
    "filename":True,
}

# add scenario 3
json_scenario['3'] = {
    "name": "baseline",
    "date_start": "2006-08-01",
    "date_end": "2006-8-31",
    "filename":False,
}

In [ ]:
df_syn = pd.DataFrame([])
filename_suffix = ''

print("############################")
# loop through each scenario and stitch each scenario together
for sce in json_scenario.keys():
    
    # define which fault was the focus on this testing
    if json_scenario[sce]['filename'] == True:
        filename_suffix = json_scenario[sce]['name']
        
    # output display
    print(f"scenario number = {sce}")
    sce_name = json_scenario[sce]["name"]
    print(f"scenario name = {sce_name}")
    
    # read simulation file based on fault type specified in scenario json
    df_temp = pd.read_csv(folder + f"/{sce_name}.csv", index_col = 0)
    
    # parse scenario name between fault type and fault intensity
    if sce_name != "baseline":
        sce_name = sce_name.rsplit("_",1)[0]
        print(f"parsed scenario name = {sce_name}")
    
    # add fault label
    df_temp["label"] = sce_name
    
    # check data size
    print(f"data shape = {df_temp.shape}")
    
    # filter data based date range specified in scenario json
    df_temp.index = pd.to_datetime(df_temp.index)
    date_start = json_scenario[sce]['date_start']
    date_end = json_scenario[sce]['date_end']
    df_temp = df_temp[date_start:date_end]
    
    # concat each scenario into one dataframe
    df_syn = pd.concat([df_syn, df_temp.reset_index()], ignore_index=True)
    
    print("############################")
    
# save to file
df_syn = df_syn.set_index("OS_time")
df_syn.to_csv(dir_save + "/stream_data_preformatted_{}.csv".format(filename_suffix))
df_syn